In [1]:
pwd

'/home/share/huadjyin/home/zhoumin3/zhoumin/benchmark_data/01A_total_re/07grn'

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import pickle 
import sys
sys.path.append('/home/share/huadjyin/home/zhoumin3/zhoumin/gears/GEARS_misc/legacy')
from model import linear_model
from utils import parse_any_pert
from tqdm import tqdm
#from data import PertDataloader
#sys.path.append('/home/share/huadjyin/home/fengtiannan/zhoumin/gears/GEARS_misc/gears')
from gears import PertData, GEARS
import torch
from gears.inference import evaluate, compute_metrics, deeper_analysis, non_dropout_analysis, non_zero_analysis
import warnings
warnings.filterwarnings("ignore")
import argparse
#seed = 5

## 1

In [ ]:
import os
import scanpy as sc
graph_type = 'grnboost'
names = ['Dixit_combined']

for name in names:
    name_lower = name.lower()
    data_path = f'./{name_lower}/perturb_processed.h5ad'
    adata = sc.read_h5ad(data_path)
    adata.var['gene_symbols'] = adata.var['gene_name']
    pert_data = PertData('./')
    data_name = f'./{name_lower}'
    data_path = os.path.join('./', data_name)
    pert_data.load(data_name=data_name, data_path=data_path)
    for split_num in range(3,6):
        pert_data.prepare_split(split='simulation', seed=split_num)
        pert_data.get_dataloader(batch_size=64, test_batch_size=64)
    
        split_path = f"./{data_name}/splits/{data_name}_simulation_{split_num}_0.75.pkl"
        import pickle 
        split = pickle.load(open(split_path, 'rb'))
        
        condition2set = {}
            
        for i,j in split.items():
            for k in j:
                condition2set[k] = i
                
        graph_path = f'./01scenic_adj/{data_name}_adjacencies_{split_num}.csv'
        weights_path = f'./03learn_weights/{data_name}/{data_name}_split{split_num}_top50_linear_learntweights.csv'
        
        gene_list = adata.var.gene_name.values
        
        model = linear_model(graph_path=graph_path, 
                     weights_path=weights_path, 
                     gene_list = gene_list,
                     binary=False, 
                     pos_edges=False, 
                     hops=1,
                     species='human')
        pred_delta = {pert: model.simulate_pert(parse_any_pert(pert)) for pert in split['test']}
        adata_ctrl = adata[adata.obs.condition == 'ctrl']
        pert_cat = []
        pred = []
        truth = []
        pred_de = []
        truth_de = []
        results = {}
        gene_ids_de = [] # zhoumin
        
        for batch in tqdm(pert_data.dataloader['test_loader']):
            
            pert_cat.extend(batch.pert)
            p = np.array([pred_delta[i]+adata_ctrl.X[np.random.randint(0, adata_ctrl.shape[0])].toarray().reshape(-1,) for i in batch.pert])
            t = batch.y
        
            pred.extend(p)
            truth.extend(t.cpu())
    
            for itr, de_idx in enumerate(batch.de_idx):
                pred_de.append(p[itr, de_idx])
                truth_de.append(t[itr, de_idx])
                gene_names_de = [pert_data.adata.var_names[i] for i in de_idx]
                gene_ids_de.append(gene_names_de)
    
            
        # all genes
        results['pert_cat'] = np.array(pert_cat)
        
        pred = np.stack(pred)
        truth = torch.stack(truth)
        results['pred']= pred
        results['truth']= truth.detach().cpu().numpy()
        
        
        pred_de = np.stack(pred_de)
        truth_de = torch.stack(truth_de)
        results['pred_de']= pred_de
        results['truth_de']= truth_de.detach().cpu().numpy()
        results["gene_ids_de"] = gene_ids_de
        from pathlib import Path
        save_dir = Path(f"/home/share/huadjyin/home/zhoumin3/zhoumin/model_benchmark/01_A_results/{name}/grn/split{split_num}/")
        save_dir.mkdir(parents=True, exist_ok=True)
        print(f"saving to {save_dir}")
        import pickle
        with open(f'{save_dir}/{name}_split{split_num}_test_res.pkl', 'wb') as f:
            pickle.dump(results, f)
        test_res = results
        test_metrics, test_pert_res = compute_metrics(test_res)
        deeper_res = deeper_analysis(pert_data.adata, test_res)
        non_dropout_res = non_dropout_analysis(pert_data.adata, test_res)
        non_zero_res = non_zero_analysis(pert_data.adata, test_res)
        
        import pickle
        with open(f'{save_dir}/{name}_split{split_num}_test_metrics.pkl', 'wb') as f:
            pickle.dump(test_metrics, f)
        with open(f'{save_dir}/{name}_split{split_num}_test_pert_res.pkl', 'wb') as f:
            pickle.dump(test_pert_res, f)
        with open(f'{save_dir}/{name}_split{split_num}_deeper_res.pkl', 'wb') as f:
            pickle.dump(deeper_res, f)
        with open(f'{save_dir}/{name}_split{split_num}_non_dropout_res.pkl', 'wb') as f:
            pickle.dump(non_dropout_res, f)
        with open(f'{save_dir}/{name}_split{split_num}_non_zero_res.pkl', 'wb') as f:
            pickle.dump(non_zero_res, f)
        
        
    print(f'{name} finished')

Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
[]
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:6
Done!
Creating dataloaders....
Done!


here1
Edges: 493742


100%|█████████████████████████████████████████| 150/150 [04:05<00:00,  1.64s/it]


saving to /home/share/huadjyin/home/zhoumin3/zhoumin/model_benchmark/01_A_results/Dixit_combined/grn/split3


Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:6
Done!
Creating dataloaders....
Done!


...

saving to /home/share/huadjyin/home/zhoumin3/zhoumin/model_benchmark/01_A_results/Dixit_combined/grn/split4


Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:6
Done!
Creating dataloaders....
Done!


here1
Edges: 493742


100%|█████████████████████████████████████████| 147/147 [03:38<00:00,  1.48s/it]


saving to /home/share/huadjyin/home/zhoumin3/zhoumin/model_benchmark/01_A_results/Dixit_combined/grn/split5
Dixit_combined finished


## replogle_k562_essential

In [ ]:
import os
import scanpy as sc
graph_type = 'grnboost'
names = ['Replogle_k562_essential']

for name in names:
    name_lower = name.lower()
    data_path = f'./{name_lower}/perturb_processed.h5ad'
    adata = sc.read_h5ad(data_path)
    adata.var['gene_symbols'] = adata.var['gene_name']
    pert_data = PertData('./')
    data_name = f'./{name_lower}'
    data_path = os.path.join('./', data_name)
    pert_data.load(data_name=data_name, data_path=data_path)
    for split_num in range(1,2):
        pert_data.prepare_split(split='simulation', seed=split_num)
        pert_data.get_dataloader(batch_size=64, test_batch_size=64)
    
        split_path = f"./{data_name}/splits/{data_name}_simulation_{split_num}_0.75.pkl"
        import pickle 
        split = pickle.load(open(split_path, 'rb'))
        
        condition2set = {}
            
        for i,j in split.items():
            for k in j:
                condition2set[k] = i
                
        graph_path = f'./01scenic_adj/{data_name}_adjacencies_{split_num}.csv'
        weights_path = f'./03learn_weights/{data_name}/{data_name}_split{split_num}_top50_linear_learntweights.csv'
        
        gene_list = adata.var.gene_name.values
        
        model = linear_model(graph_path=graph_path, 
                     weights_path=weights_path, 
                     gene_list = gene_list,
                     binary=False, 
                     pos_edges=False, 
                     hops=1,
                     species='human')
        pred_delta = {pert: model.simulate_pert(parse_any_pert(pert)) for pert in split['test']}
        adata_ctrl = adata[adata.obs.condition == 'ctrl']
        pert_cat = []
        pred = []
        truth = []
        pred_de = []
        truth_de = []
        results = {}
        gene_ids_de = [] # zhoumin
        
        for batch in tqdm(pert_data.dataloader['test_loader']):
            
            pert_cat.extend(batch.pert)
            p = np.array([pred_delta[i]+adata_ctrl.X[np.random.randint(0, adata_ctrl.shape[0])].toarray().reshape(-1,) for i in batch.pert])
            t = batch.y
        
            pred.extend(p)
            truth.extend(t.cpu())
    
            for itr, de_idx in enumerate(batch.de_idx):
                pred_de.append(p[itr, de_idx])
                truth_de.append(t[itr, de_idx])
                gene_names_de = [pert_data.adata.var_names[i] for i in de_idx]
                gene_ids_de.append(gene_names_de)
    
            
        # all genes
        results['pert_cat'] = np.array(pert_cat)
        
        pred = np.stack(pred)
        truth = torch.stack(truth)
        results['pred']= pred
        results['truth']= truth.detach().cpu().numpy()
        
        
        pred_de = np.stack(pred_de)
        truth_de = torch.stack(truth_de)
        results['pred_de']= pred_de
        results['truth_de']= truth_de.detach().cpu().numpy()
        results["gene_ids_de"] = gene_ids_de
        from pathlib import Path
        save_dir = Path(f"/home/share/huadjyin/home/zhoumin3/zhoumin/model_benchmark/01_A_results/{name}/grn/split{split_num}/")
        save_dir.mkdir(parents=True, exist_ok=True)
        print(f"saving to {save_dir}")
        import pickle
        # 保存为.pkl文件
        with open(f'{save_dir}/{name}_split{split_num}_test_res.pkl', 'wb') as f:
            pickle.dump(results, f)
        test_res = results
        test_metrics, test_pert_res = compute_metrics(test_res)
        deeper_res = deeper_analysis(pert_data.adata, test_res)
        non_dropout_res = non_dropout_analysis(pert_data.adata, test_res)
        non_zero_res = non_zero_analysis(pert_data.adata, test_res)
        
        import pickle
        with open(f'{save_dir}/{name}_split{split_num}_test_metrics.pkl', 'wb') as f:
            pickle.dump(test_metrics, f)
        with open(f'{save_dir}/{name}_split{split_num}_test_pert_res.pkl', 'wb') as f:
            pickle.dump(test_pert_res, f)
        with open(f'{save_dir}/{name}_split{split_num}_deeper_res.pkl', 'wb') as f:
            pickle.dump(deeper_res, f)
        with open(f'{save_dir}/{name}_split{split_num}_non_dropout_res.pkl', 'wb') as f:
            pickle.dump(non_dropout_res, f)
        with open(f'{save_dir}/{name}_split{split_num}_non_zero_res.pkl', 'wb') as f:
            pickle.dump(non_zero_res, f)
        
        
    print(f'{name} finished')

Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
[]
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:266
Done!
Creating dataloaders....
Done!


here1
Edges: 493742


100%|█████████████████████████████████████████| 561/561 [22:31<00:00,  2.41s/it]


saving to /home/share/huadjyin/home/zhoumin3/zhoumin/model_benchmark/01_A_results/Replogle_k562_essential/grn/split1
Replogle_k562_essential finished


## replogle_rpe1_essential

In [ ]:
import os
import scanpy as sc
graph_type = 'grnboost'
names = ['Replogle_rpe1_essential']

for name in names:
    name_lower = name.lower()
    data_path = f'./{name_lower}/perturb_processed.h5ad'
    adata = sc.read_h5ad(data_path)
    adata.var['gene_symbols'] = adata.var['gene_name']
    pert_data = PertData('./')
    data_name = f'./{name_lower}'
    data_path = os.path.join('./', data_name)
    pert_data.load(data_name=data_name, data_path=data_path)
    for split_num in range(2,3):
        pert_data.prepare_split(split='simulation', seed=split_num)
        pert_data.get_dataloader(batch_size=64, test_batch_size=64)
    
        split_path = f"./{data_name}/splits/{data_name}_simulation_{split_num}_0.75.pkl"
        import pickle 
        split = pickle.load(open(split_path, 'rb'))
        
        condition2set = {}
            
        for i,j in split.items():
            for k in j:
                condition2set[k] = i
                
        graph_path = f'./01scenic_adj/{data_name}_adjacencies_{split_num}.csv'
        weights_path = f'./03learn_weights/{data_name}/{data_name}_split{split_num}_top50_linear_learntweights.csv'
        
        gene_list = adata.var.gene_name.values
        
        model = linear_model(graph_path=graph_path, 
                     weights_path=weights_path, 
                     gene_list = gene_list,
                     binary=False, 
                     pos_edges=False, 
                     hops=1,
                     species='human')
        pred_delta = {pert: model.simulate_pert(parse_any_pert(pert)) for pert in split['test']}
        adata_ctrl = adata[adata.obs.condition == 'ctrl']
        pert_cat = []
        pred = []
        truth = []
        pred_de = []
        truth_de = []
        results = {}
        gene_ids_de = [] # zhoumin
        
        for batch in tqdm(pert_data.dataloader['test_loader']):
            
            pert_cat.extend(batch.pert)
            p = np.array([pred_delta[i]+adata_ctrl.X[np.random.randint(0, adata_ctrl.shape[0])].toarray().reshape(-1,) for i in batch.pert])
            t = batch.y
        
            pred.extend(p)
            truth.extend(t.cpu())
    
            for itr, de_idx in enumerate(batch.de_idx):
                pred_de.append(p[itr, de_idx])
                truth_de.append(t[itr, de_idx])
                gene_names_de = [pert_data.adata.var_names[i] for i in de_idx]
                gene_ids_de.append(gene_names_de)
    
            
        # all genes
        results['pert_cat'] = np.array(pert_cat)
        
        pred = np.stack(pred)
        truth = torch.stack(truth)
        results['pred']= pred
        results['truth']= truth.detach().cpu().numpy()
        
        
        pred_de = np.stack(pred_de)
        truth_de = torch.stack(truth_de)
        results['pred_de']= pred_de
        results['truth_de']= truth_de.detach().cpu().numpy()
        results["gene_ids_de"] = gene_ids_de
        from pathlib import Path
        save_dir = Path(f"/home/share/huadjyin/home/zhoumin3/zhoumin/model_benchmark/01_A_results/{name}/grn/split{split_num}/")
        save_dir.mkdir(parents=True, exist_ok=True)
        print(f"saving to {save_dir}")
        import pickle
        with open(f'{save_dir}/{name}_split{split_num}_test_res.pkl', 'wb') as f:
            pickle.dump(results, f)
        test_res = results
        test_metrics, test_pert_res = compute_metrics(test_res)
        deeper_res = deeper_analysis(pert_data.adata, test_res)
        non_dropout_res = non_dropout_analysis(pert_data.adata, test_res)
        non_zero_res = non_zero_analysis(pert_data.adata, test_res)
        
        import pickle
        with open(f'{save_dir}/{name}_split{split_num}_test_metrics.pkl', 'wb') as f:
            pickle.dump(test_metrics, f)
        with open(f'{save_dir}/{name}_split{split_num}_test_pert_res.pkl', 'wb') as f:
            pickle.dump(test_pert_res, f)
        with open(f'{save_dir}/{name}_split{split_num}_deeper_res.pkl', 'wb') as f:
            pickle.dump(deeper_res, f)
        with open(f'{save_dir}/{name}_split{split_num}_non_dropout_res.pkl', 'wb') as f:
            pickle.dump(non_dropout_res, f)
        with open(f'{save_dir}/{name}_split{split_num}_non_zero_res.pkl', 'wb') as f:
            pickle.dump(non_zero_res, f)
        
        
    print(f'{name} finished')

Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
[]
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:360
Done!
Creating dataloaders....
Done!


here1
Edges: 493742


100%|█████████████████████████████████████████| 516/516 [19:20<00:00,  2.25s/it]


saving to /home/share/huadjyin/home/zhoumin3/zhoumin/model_benchmark/01_A_results/Replogle_rpe1_essential/grn/split2


KeyError: 'AC118549.1+ctrl'